<a href="https://colab.research.google.com/github/danielhou13/cogs402longformer/blob/main/fakeclinicalnotes/src/cogs402projecttestnotes/T3-vis_head_importance_notes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is a part of the [T3-vis](https://arxiv.org/abs/2108.13587) implmentation for visualizing Transformer Neural Networks. Here, using the dataset, model and the functions, we calculate the importance of each head in each layer, allowing us to scale the attention output of a transformer model (in this case longformer) by their head and layer. By doing so, there is more attention on the tokens for the head and layer with higher importance. It operates on the validation set and we export the results for use in other notebooks.

### Import Packages

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import torch
import numpy as np
from tqdm import tqdm
import pdb

In [ ]:
pip install datasets --quiet

     |████████████████████████████████| 365 kB 14.7 MB/s 
     |████████████████████████████████| 101 kB 10.4 MB/s 
     |████████████████████████████████| 212 kB 66.0 MB/s 
     |████████████████████████████████| 140 kB 67.8 MB/s 
     |████████████████████████████████| 1.1 MB 60.9 MB/s 
     |████████████████████████████████| 596 kB 64.8 MB/s 
     |████████████████████████████████| 127 kB 84.2 MB/s 
     |████████████████████████████████| 144 kB 69.5 MB/s 
     |████████████████████████████████| 271 kB 68.3 MB/s 
     |████████████████████████████████| 94 kB 3.4 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
pip install transformers --quiet

     |████████████████████████████████| 4.4 MB 15.2 MB/s 
     |████████████████████████████████| 6.6 MB 47.5 MB/s 


### Import dataset

Here we are importing the model and the dataset we want to assess. The import is replicating the manner used by the T3-vis implementation, with the removal of a few items such as "idx" and "visualize columns" as they are unnecessary. 

In [ ]:
from datasets import load_dataset
from transformers import LongformerForSequenceClassification, AutoTokenizer

def longformer_finetuned_papers():
    test = torch.load("/content/drive/MyDrive/cogs402longformer/fakeclinicalnotes/models/full_augmented_lr2e-5_dropout3_10_trained_threshold.pt")
    model = LongformerForSequenceClassification.from_pretrained('allenai/longformer-base-4096', state_dict=test['state_dict'], num_labels = 2)
    return model

def preprocess_function(tokenizer, example, max_length):
    example.update(tokenizer(example['text'], padding='max_length', max_length=max_length, truncation=True))
    return example

def get_papers_dataset(dataset_type):
    max_length = 4096
    dataset = load_dataset("danielhou13/cogs402datafake")[dataset_type]
    new_col = list(np.arange(0, len(dataset)))

    tokenizer = AutoTokenizer.from_pretrained('allenai/longformer-base-4096')

    dataset = dataset.map(lambda x: preprocess_function(tokenizer, x, max_length), batched=True)
    setattr(dataset, 'input_columns', ['input_ids', 'attention_mask'])
    setattr(dataset, 'target_columns', ['labels'])
    setattr(dataset, 'max_length', max_length)
    setattr(dataset, 'tokenizer', tokenizer)
    return dataset

def papers_train_set():
    return get_papers_dataset('train')

def papers_test_set():
    return get_papers_dataset('test')

### T3-vis functions

These functions are a copy of the T3-vis functions but substituting bert for longformer. 

We iterate over the entire dataset and compute the importance of each head for every layer.

**Output:
Array of shape: (Layer, Head)**

Each item in the array will be (after normalization) a value from 0.0 - 1.0 indicating how important that particular head is (1.0 being most important and 0.0 being least).


In [ ]:
def normalize(matrix, axis=None):
    normalized = (matrix - matrix.min(axis=axis)) /\
                 (matrix.max(axis=axis) - matrix.min(axis=axis))
    return normalized

def find_pruneable_heads_and_indices(heads, n_heads, head_size, already_pruned_heads):
    """
    List, int, int, set -> Tuple[set, "torch.LongTensor"]
    """

    mask = torch.ones(n_heads, head_size)
    heads = set(heads) - already_pruned_heads  # Convert to set and remove already pruned heads
    for head in heads:
        # Compute how many pruned heads are before the head and move the index accordingly
        head = head - sum(1 if h < head else 0 for h in already_pruned_heads)
        mask[head] = 0

    mask = mask.view(-1).contiguous().eq(1)
    index: torch.LongTensor = torch.arange(len(mask))[~mask].long()
    return heads, index

def get_taylor_importance(model):
    n_layers = model.config.num_hidden_layers
    n_heads = model.config.num_attention_heads
    head_size = int(model.config.hidden_size / n_heads)
    importance_scores = np.zeros((n_layers, n_heads))

    for i in range(n_layers):
        attention = model.longformer.encoder.layer[i].attention
        num_attention_heads = attention.self.num_heads

        pruned_heads = attention.pruned_heads
        leftover_heads = set(list(range(n_heads))) - pruned_heads

        for head_idx in leftover_heads:
            heads, index = find_pruneable_heads_and_indices([head_idx], num_attention_heads, head_size, pruned_heads)
            index = index.to(model.device)

            query_b_grad = (attention.self.query.bias.grad[index] *\
                            attention.self.query.bias[index]) ** 2
            query_W_grad = (attention.self.query.weight.grad.index_select(0, index) *\
                            attention.self.query.weight.index_select(0, index)) ** 2

            key_b_grad = (attention.self.key.bias.grad[index] *\
                          attention.self.key.bias[index]) ** 2
            key_W_grad = (attention.self.key.weight.grad.index_select(0, index) *\
                          attention.self.key.weight.index_select(0, index)) ** 2

            value_b_grad = (attention.self.value.bias.grad[index] *\
                            attention.self.value.bias[index]) ** 2
            value_W_grad = (attention.self.value.weight.grad.index_select(0, index) *\
                            attention.self.value.weight.index_select(0, index)) ** 2

            output_W_grad = (attention.output.dense.weight.grad.index_select(1, index) *
                             attention.output.dense.weight.index_select(1, index)) ** 2
            abs_grad_magnitude = query_b_grad.sum() + query_W_grad.sum() + key_b_grad.sum() + \
                key_W_grad.sum() + value_b_grad.sum() + value_W_grad.sum() + output_W_grad.sum()

                
            score = abs_grad_magnitude.item()
            importance_scores[i, head_idx] += score
    return importance_scores


def compute_importance(model, dataloader, measure='taylor'):

    assert measure in ['taylor', 'oracle', 'sensitivity']

    max_input_len = model.config.max_position_embeddings
    n_layers = model.config.num_hidden_layers
    n_heads = model.config.num_attention_heads
    head_size = int(model.config.hidden_size / n_heads)

    importance_scores = np.zeros((n_layers, n_heads))

    device = model.device
    total_loss = 0.

    if measure == 'sensitivity':
        head_mask = torch.ones(n_layers, n_heads).to(device)
        head_mask.requires_grad_(requires_grad=True)
    else:
        head_mask = None

    for step, inputs in enumerate(tqdm(dataloader)):
        batch_size_ = inputs['input_ids'].__len__()

        if torch.cuda.is_available():
            for k, v in inputs.items():
                if isinstance(v, torch.Tensor):
                    inputs[k] = v.cuda()


        output = model(**inputs)
        loss = output['loss']
        loss.backward()

        if measure == 'sensitivity':
            importance_scores += head_mask.grad.abs().detach().cpu().numpy()
        elif measure == 'taylor':
            importance_scores = get_taylor_importance(model)

    return importance_scores

Here we are calling the functions to import the dataset, model and making sure that the dataset is in a pytorch compatible manner.

In [ ]:
cogs402_test = papers_train_set()
model = longformer_finetuned_papers()
columns = cogs402_test.input_columns + cogs402_test.target_columns
print(columns)
cogs402_test.set_format(type='torch', columns=columns)
cogs402_test=cogs402_test.remove_columns(['text'])
print(cogs402_test)

Downloading:   0%|          | 0.00/613 [00:00<?, ?B/s]

Using custom data configuration danielhou13--cogs402datafake-f5349e6cf83e41d8


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/danielhou13___parquet/danielhou13--cogs402datafake-f5349e6cf83e41d8/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


...

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Downloading:   0%|          | 0.00/570M [00:00<?, ?B/s]

Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerForSequenceClassification: ['longformer_model.embeddings.LayerNorm.weight', 'longformer_model.encoder.layer.10.attention.self.value.weight', 'longformer_model.encoder.layer.9.attention.output.dense.bias', 'fc.bias', 'longformer_model.encoder.layer.7.attention.self.query_global.weight', 'longformer_model.encoder.layer.8.attention.self.query.weight', 'longformer_model.encoder.layer.9.output.dense.bias', 'longformer_model.encoder.layer.10.attention.self.key_global.weight', 'longformer_model.encoder.layer.11.attention.self.value_global.bias', 'longformer_model.encoder.layer.11.attention.output.LayerNorm.bias', 'longformer_model.encoder.layer.4.attention.self.value_global.weight', 'longformer_model.encoder.layer.6.attention.self.key.weight', 'longformer_model.encoder.layer.0.attention.self.value_global.weight', 'longformer_model.encoder.layer.11.attention.self.value.weight', 'longf

['input_ids', 'attention_mask', 'labels']
Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 12
})


Don't forget to send the model to GPU if you have one.

In [ ]:
if torch.cuda.is_available():
    model = model.cuda()

print(model.device)

cuda:0


The functions operate using the dataloader so we convert our validation set (as per the T3-vis implementation) to a dataloader. We keep batch size to 1 to minimize the memory required as longformer models are memory intensive.

In [ ]:
dataloader = torch.utils.data.DataLoader(cogs402_test, batch_size=1)

Finally, we run the function, normalize the resulting matrix, and save the results for future use.

In [ ]:
importance = compute_importance(model, dataloader)
importance = normalize(importance)

100%|██████████| 12/12 [00:22<00:00,  1.91s/it]


Lastly, we save our importance matrix. Remember to change the path to whatever suits your project's needs. The commented-out line of code saves the importance matrix in the current working directory.


In [ ]:
torch.save(importance, "/content/drive/MyDrive/cogs402longformer/fakeclinicalnotes/t3-visapplication/notes/head_importance.pt")
# torch.save(importance, "head_importance.pt")